In [ ]:
Here we will 

In [8]:
import os
import skbio
from biom import Table, load_table
from biom.util import biom_open
from bloom import remove_seqs, trim_seqs

In [2]:
data_dir = '../data/'
results_dir = '../results'
zip_file = '30_seqs.fna.gz'
seqs_file = '30_seqs.fna'
biom_file = 'ercolini.feces.clean.withtax.biom'
filtered_file = 'filtered.fna'

zip_file = os.path.join(data_dir, zip_file)
seqs_file = os.path.join(data_dir, seqs_file)
biom_file = os.path.join(data_dir, biom_file)
filtered_file = os.path.join(results_dir, filtered_file)

In [3]:
!gunzip ../data/30_seqs.fna.gz

gunzip: can't stat: ../data/30_seqs.fna.gz (../data/30_seqs.fna.gz.gz): No such file or directory


In [4]:
seqs = skbio.read(seqs_file, format='fasta')
table = load_table(biom_file)

In [5]:
length = min(map(len, table.ids(axis='observation')))

In [6]:
seqs = trim_seqs(seqs, length=length)
outtable = remove_seqs(table, seqs)

In [9]:
with biom_open(filtered_file, 'w') as f:
        outtable.to_hdf5(f, "filterbiomseqs")
